## <span style="color:purple">Dependency syntactic analysis with UDPipe</span>

[UDPipe](http://ufal.mff.cuni.cz/udpipe) is a trainable pipeline for tokenization, tagging, lemmatization and dependency parsing of CoNLL files. 
EstNLTK provides a wrapper for UDPipe dependency parser and provides models of syntactic analysis that output UD or CG format dependency relations. 

### UDPipeTagger

#### Requirements
**UDPipe executable**. In order to use UDPipe based syntactic analysis, the UDPipe must be installed into the system. The information about how to install it can be found from [UDPipe Installation page](http://ufal.mff.cuni.cz/udpipe/install).
 
EstNLTK expects that the directory containing UDPipe executable (udpipe in UNIX, udpipe.exe in Windows) is accessible from system's environment variable PATH. UDPipe executables can be downloaded [here](https://github.com/ufal/udpipe/releases/tag/v1.2.0) (currently supported version is 1.2.0).

You can test that the UDPipe is available by the following command:

In [1]:
!udpipe --version

UDPipe version 1.2.0 (using UniLib 3.1.1,
MorphoDiTa 1.9.1-devel, Parsito 1.1.1-devel)
Copyright 2016 by Institute of Formal and Applied Linguistics, Faculty of
Mathematics and Physics, Charles University in Prague, Czech Republic.


**UDPipe's models**. As with other data-driven syntactic parsers, UDPipe's models are distributed separately and need to be downloaded: 

* If you create a new instance of `UDPipeTagger` and the models are missing, you'll be prompted with a question asking for permission to download the models;
* Alternatively, you can pre-download all models manually via `download` function:
```python
from estnltk import download
download('udpipetagger')
```

The following table gives an overview about EstNLTK's UDPipeTagger models:

|             | UD + morph_analysis | UD + morph_extended | UD + morph_extended <br> (with VislCG3 parser) | CG + morph_analysis | CG + morph_extended | CG + morph_extended <br> (with VislCG3 parser)  |
| ----------- | ------------- | --------- | -------------- | -------- | -------- | -------- |
| **model name <br>(with relative path)**  | conllu\ud_morph_analysis.output  |  conllu\ud_morph_extended.output | model_1.output | conllx\model_2.output          | conllx\model_3.output   | model_0.output |
| **Needs to be downloaded?** | Yes | Yes | Yes | Yes | Yes | Yes |
| **required preprocessing** | `words`, `sentences`, `morph_analysis`, `conll_morph` (from `morph_analysis`)  | `words`, `sentences`, `morph_extended`, `conll_morph` (from `morph_extended`) | `words`, `sentences`, `morph_extended`, `conll_morph` from `morph_extended` (requires VislCG3 parser) | `words`, `sentences`, `morph_analysis`, `conll_morph` (from `morph_analysis`) | `words`, `sentences`, `morph_extended`, `conll_morph` (from `morph_extended`) | `words`, `sentences`, `morph_extended`, `conll_morph` from `morph_extended` (requires VislCG3 parser) |
| **UDPipeTagger's <br>configuration** | input_type='morph_analysis',<br> version='conllu' | input_type='morph_extended',<br> version='conllu' | input_type='visl_morph',<br> version='conllu' | input_type='morph_analysis',<br> version='conllx' | input_type='morph_extended',<br> version='conllx' | input_type='visl_morph',<br> version='conllx' | 
| **Dependency relations (_deprel_)** | [UD tags](https://universaldependencies.org/u/dep/index.html) | [UD tags](https://universaldependencies.org/u/dep/index.html) | [UD tags](https://universaldependencies.org/u/dep/index.html) | [CG tags](https://korpused.keeleressursid.ee/syntaks/dokumendid/syntaksiliides_en.pdf) |  [CG tags](https://korpused.keeleressursid.ee/syntaks/dokumendid/syntaksiliides_en.pdf) | [CG tags](https://korpused.keeleressursid.ee/syntaks/dokumendid/syntaksiliides_en.pdf) |
| ***upostag, xpostag, feats*** | [Vabamorf's tagset](https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/B_morphology/00_tables_of_morphological_categories.ipynb) | [morph_extended tags](01_syntax_preprocessing.ipynb) | [morph_extended tags](01_syntax_preprocessing.ipynb) | [Vabamorf's tagset](https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/B_morphology/00_tables_of_morphological_categories.ipynb) | [morph_extended tags](01_syntax_preprocessing.ipynb) | [morph_extended tags](01_syntax_preprocessing.ipynb) |


#### Preprocessing for UDPipeTagger

UDPipeTagger preprocessing is analogous to [MaltparserTagger's preprocessing](03_syntactic_analysis_with_maltparser.ipynb).
For models not requiring VislCG3 parser, you can use `ConllMorphTagger` with the setting `no_visl=True` for preprocessing. 
This transforms annotations of the preprocessing layer to CONLL format fields, as described [here](01_syntax_preprocessing.ipynb). 
The parameter `morph_extended_layer` can be used to change between input layers `morph_analysis` and `morph_extended`.

In [2]:
from estnltk.taggers import ConllMorphTagger

# create preprocessing tagger
conll_tagger = ConllMorphTagger(output_layer='conll_morph',       # default: 'conll_morph'
                          morph_extended_layer='morph_analysis',  # default: 'morph_extended'
                          no_visl=True
                          )

# Create text and preprocess for UDPipeTagger syntax
from estnltk import Text
text = Text("Tema tahtis saada rikkaks ja kuulsaks ja elada kodanlikku elu .")
text.tag_layer('morph_analysis')
conll_tagger.tag(text)

Text(text='Tema tahtis saada rikkaks ja kuulsaks ja elada kodanlikku elu .')

For models that require VislCG3 parser ( model_1 and model_0 ), use `ConllMorphTagger` with the setting `no_visl=False` (which is default, by the way). 
Note, however, that this requires that **VISLCG3 parser is installed into the system, and accessible from system's environment variable PATH** .
The information about the parser is distributed in the [Constraint Grammar's Google Group](https://groups.google.com/g/constraint-grammar), and this is also the place to look for the most compact guide about [getting & installing the parser](https://groups.google.com/g/constraint-grammar/c/fNMkpAb_g3U).
Once you have the parser installed and available, you can create a suitable ConllMorphTagger in the following way:

```python
# Initialize ConllMorphTagger with VislCG3 preprocessing (only works for 'morph_extended')
conll_tagger = ConllMorphTagger(morph_extended_layer='morph_extended')
```

In [3]:
import warnings
warnings.filterwarnings("ignore")
#
# Ignore the following repeated warnings:
#
# ..\lib\subprocess.py:935: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
#  self.stdin = io.open(p2cwrite, 'wb', bufsize)
# ..\lib\subprocess.py:941: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
#  self.stdout = io.open(c2pread, 'rb', bufsize)
# ..\lib\subprocess.py:946: RuntimeWarning: line buffering (buffering=1) isn't supported in binary mode, the default buffer size will be used
#  self.stderr = io.open(errread, 'rb', bufsize)

#### Basic usage

You can change UDPipeTagger's model via combination of parameters `version` and `input_type`.
Parameter `version='conllu'` specifies to use a model with the UD deprel tags (default), and `version='conllx'` a model with the CG deprel tags.
Parameter `input_type` specifies the input preprocessing layer, possible values: `input_type='morph_analysis'`,  `input_type='morph_extended'`, and `input_type='visl_morph'` (morph_extended processed with VislCG3 parser; default).

In [4]:
from estnltk.taggers import UDPipeTagger

udpipe_tagger = UDPipeTagger(input_type='morph_analysis', version='conllx')

udpipe_tagger.tag(text)

text.udpipe_syntax

Layer(name='udpipe_syntax', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Tema', [{'id': 1, 'form': 'Tema', 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('sg', ''), ('n', '')]), 'head': 2, 'deprel': '@SUBJ', 'deps': '_', 'misc': '_'}]),
Span('tahtis', [{'id': 2, 'form': 'tahtis', 'lemma': 'tahtma', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('s', '')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_'}]),
Span('saada', [{'id': 3, 'form': 'saada', 'lemma': 'saama', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('da', '')]), 'head': 2, 'deprel': '@OBJ', 'deps': '_', 'misc': '_'}]),
Span('rikkaks', [{'id': 4, 'form': 'rikkaks', 'lemma': 'rikas', 'upostag': 'A', 'xpostag': 'A', 'feats': OrderedDict([('sg', ''), ('tr', '')]), 'head': 3, 'deprel': '@ADVL', 'deps': '_', 'misc': '_'}]),
Span('ja', [{'id': 5, 'form': 'ja', 'lemma': 'ja', 'upostag': 'J', 'xpostag': 'J', 'feats': OrderedDict(), 'head': 6, 'deprel': '@J', 'deps': '_', 'misc': '_'}]),
Span('kuulsaks', [{'id': 6, 'form': 'kuulsaks', 'lemma': 'kuulus', 'upostag': 'A', 'xpostag': 'A', 'feats': OrderedDict([('sg', ''), ('tr', '')]), 'head': 4, 'deprel': '@ADVL', 'deps': '_', 'misc': '_'}]),
Span('ja', [{'id': 7, 'form': 'ja', 'lemma': 'ja', 'upostag': 'J', 'xpostag': 'J', 'feats': OrderedDict(), 'head': 8, 'deprel': '@J', 'deps': '_', 'misc': '_'}]),
Span('elada', [{'id': 8, 'form': 'elada', 'lemma': 'elama', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('da', '')]), 'head': 3, 'deprel': '@OBJ', 'deps': '_', 'misc': '_'}]),
Span('kodanlikku', [{'id': 9, 'form': 'kodanlikku', 'lemma': 'kodanlik', 'upostag': 'A', 'xpostag': 'A', 'feats': OrderedDict([('sg', ''), ('p', '')]), 'head': 10, 'deprel': '@AN>', 'deps': '_', 'misc': '_'}]),
Span('elu', [{'id': 10, 'form': 'elu', 'lemma': 'elu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('p', '')]), 'head': 8, 'deprel': '@OBJ', 'deps': '_', 'misc': '_'}]),
Span('.', [{'id': 11, 'form': '.', 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': '@Punc', 'deps': '_', 'misc': '_'}])])

In [5]:
# Use UD dependency relations instead 
udpipe_tagger = UDPipeTagger(output_layer='udpipe_syntax_ud', input_type='morph_analysis', version='conllu')

udpipe_tagger.tag(text)

text.udpipe_syntax_ud

Layer(name='udpipe_syntax_ud', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Tema', [{'id': 1, 'form': 'Tema', 'lemma': 'tema', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('sg', ''), ('n', '')]), 'head': 2, 'deprel': 'nsubj', 'deps': '_', 'misc': '_'}]),
Span('tahtis', [{'id': 2, 'form': 'tahtis', 'lemma': 'tahtma', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('s', '')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_'}]),
Span('saada', [{'id': 3, 'form': 'saada', 'lemma': 'saama', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('da', '')]), 'head': 2, 'deprel': 'xcomp', 'deps': '_', 'misc': '_'}]),
Span('rikkaks', [{'id': 4, 'form': 'rikkaks', 'lemma': 'rikas', 'upostag': 'A', 'xpostag': 'A', 'feats': OrderedDict([('sg', ''), ('tr', '')]), 'head': 3, 'deprel': 'xcomp', 'deps': '_', 'misc': '_'}]),
Span('ja', [{'id': 5, 'form': 'ja', 'lemma': 'ja', 'upostag': 'J', 'xpostag': 'J', 'feats': OrderedDict(), 'head': 6, 'deprel': 'cc', 'deps': '_', 'misc': '_'}]),
Span('kuulsaks', [{'id': 6, 'form': 'kuulsaks', 'lemma': 'kuulus', 'upostag': 'A', 'xpostag': 'A', 'feats': OrderedDict([('sg', ''), ('tr', '')]), 'head': 4, 'deprel': 'conj', 'deps': '_', 'misc': '_'}]),
Span('ja', [{'id': 7, 'form': 'ja', 'lemma': 'ja', 'upostag': 'J', 'xpostag': 'J', 'feats': OrderedDict(), 'head': 8, 'deprel': 'cc', 'deps': '_', 'misc': '_'}]),
Span('elada', [{'id': 8, 'form': 'elada', 'lemma': 'elama', 'upostag': 'V', 'xpostag': 'V', 'feats': OrderedDict([('da', '')]), 'head': 3, 'deprel': 'conj', 'deps': '_', 'misc': '_'}]),
Span('kodanlikku', [{'id': 9, 'form': 'kodanlikku', 'lemma': 'kodanlik', 'upostag': 'A', 'xpostag': 'A', 'feats': OrderedDict([('sg', ''), ('p', '')]), 'head': 10, 'deprel': 'amod', 'deps': '_', 'misc': '_'}]),
Span('elu', [{'id': 10, 'form': 'elu', 'lemma': 'elu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', ''), ('p', '')]), 'head': 8, 'deprel': 'obj', 'deps': '_', 'misc': '_'}]),
Span('.', [{'id': 11, 'form': '.', 'lemma': '.', 'upostag': 'Z', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 2, 'deprel': 'punct', 'deps': '_', 'misc': '_'}])])

**Extra flags.** Use flag `add_parent_and_children` to add extra attributes `parent_span` and `children` to the output layer, which make querying dependency relations easier. 

**Augmenting existing CoNNL-U format text.** It is also possible to add syntactic analysis to text with CoNNL-U format layer. We can read file in CoNNL-U format and add syntactic analysis to it using UDPipeTagger. To do that, version should be specified for the tagger.  

In [6]:
from estnltk.converters.conll.conll_importer import conll_to_text

text = conll_to_text(file='example.conllu', syntax_layer='conllu_morph') 

udpipe_tagger = UDPipeTagger(input_syntax_layer='conllu_morph', version='conllu') # default version is conllx

udpipe_tagger.tag(text)
text.udpipe_syntax

Layer(name='udpipe_syntax', attributes=('id', 'form', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc'), spans=SL[Span('Aga', [{'id': 1, 'form': 'Aga', 'lemma': 'aga', 'upostag': 'CCONJ', 'xpostag': 'J', 'feats': OrderedDict(), 'head': 5, 'deprel': 'cc', 'deps': '_', 'misc': '_'}]),
Span('ma', [{'id': 2, 'form': 'ma', 'lemma': 'mina', 'upostag': 'PRON', 'xpostag': 'P', 'feats': OrderedDict([('Case', 'Nom'), ('Number', 'Sing'), ('Person', '1'), ('PronType', 'Prs')]), 'head': 5, 'deprel': 'nsubj', 'deps': '_', 'misc': '_'}]),
Span('peaksin', [{'id': 3, 'form': 'peaksin', 'lemma': 'pidama', 'upostag': 'AUX', 'xpostag': 'V', 'feats': OrderedDict([('Mood', 'Cnd'), ('Number', 'Sing'), ('Person', '1'), ('Tense', 'Pres'), ('VerbForm', 'Fin'), ('Voice', 'Act')]), 'head': 5, 'deprel': 'aux', 'deps': '_', 'misc': '_'}]),
Span('vist', [{'id': 4, 'form': 'vist', 'lemma': 'vist', 'upostag': 'ADV', 'xpostag': 'D', 'feats': OrderedDict(), 'head': 5, 'deprel': 'advmod', 'deps': '_', 'misc': '_'}]),
Span('pihtima', [{'id': 5, 'form': 'pihtima', 'lemma': 'pihtima', 'upostag': 'VERB', 'xpostag': 'V', 'feats': OrderedDict([('Case', 'Ill'), ('VerbForm', 'Sup'), ('Voice', 'Act')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_'}]),
Span(',', [{'id': 6, 'form': ',', 'lemma': ',', 'upostag': 'PUNCT', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 10, 'deprel': 'punct', 'deps': '_', 'misc': '_'}]),
Span('et', [{'id': 7, 'form': 'et', 'lemma': 'et', 'upostag': 'SCONJ', 'xpostag': 'J', 'feats': OrderedDict(), 'head': 10, 'deprel': 'mark', 'deps': '_', 'misc': '_'}]),
Span('ma', [{'id': 8, 'form': 'ma', 'lemma': 'mina', 'upostag': 'PRON', 'xpostag': 'P', 'feats': OrderedDict([('Case', 'Nom'), ('Number', 'Sing'), ('Person', '1'), ('PronType', 'Prs')]), 'head': 10, 'deprel': 'nsubj', 'deps': '_', 'misc': '_'}]),
Span('ei', [{'id': 9, 'form': 'ei', 'lemma': 'ei', 'upostag': 'AUX', 'xpostag': 'V', 'feats': OrderedDict([('Polarity', 'Neg')]), 'head': 10, 'deprel': 'aux', 'deps': '_', 'misc': '_'}]),
Span('tea', [{'id': 10, 'form': 'tea', 'lemma': 'teadma', 'upostag': 'VERB', 'xpostag': 'V', 'feats': OrderedDict([('Connegative', 'Yes'), ('Mood', 'Ind'), ('Tense', 'Pres'), ('VerbForm', 'Fin'), ('Voice', 'Act')]), 'head': 5, 'deprel': 'ccomp', 'deps': '_', 'misc': '_'}]),
Span(',', [{'id': 11, 'form': ',', 'lemma': ',', 'upostag': 'PUNCT', 'xpostag': 'Z', 'feats': OrderedDict(), 'head': 17, 'deprel': 'punct', 'deps': '_', 'misc': '_'}]),
Span('mida', [{'id': 12, 'form': 'mida', 'lemma': 'mis', 'upostag': 'PRON', 'xpostag': 'P', 'feats': OrderedDict([('Case', 'Par'), ('Number', 'Sing'), ('PronType', 'Int,Rel')]), 'head': 17, 'deprel': 'obl', 'deps': '_', 'misc': '_'}]),
Span('interaktiivne', [{'id': 13, 'form': 'interaktiivne', 'lemma': 'inter_aktiivne', 'upostag': 'ADJ', 'xpostag': 'A', 'feats': OrderedDict([('Case', 'Nom'), ('Degree', 'Pos'), ('Number', 'Sing')]), 'head': 14, 'deprel': 'amod', 'deps': '_', 'misc': '_'}]),
Span('meedia', [{'id': 14, 'form': 'meedia', 'lemma': 'meedia', 'upostag': 'NOUN', 'xpostag': 'S', 'feats': OrderedDict([('Case', 'Nom'), ('Number', 'Sing')]), 'head': 17, 'deprel': 'nmod', 'deps': '_', 'misc': '_'}]),
Span('endast', [{'id': 15, 'form': 'endast', 'lemma': 'ise', 'upostag': 'PRON', 'xpostag': 'P', 'feats': OrderedDict([('Case', 'Ela'), ('Number', 'Sing'), ('PronType', 'Prs'), ('Reflex', 'Yes')]), 'head': 17, 'deprel': 'obl', 'deps': '_', 'misc': '_'}]),
Span('täpselt', [{'id': 16, 'form': 'täpselt', 'lemma': 'täpselt', 'upostag': 'ADV', 'xpostag': 'D', 'feats': OrderedDict(), 'head': 17, 'deprel': 'advmod', 'deps': '_', 'misc': '_'}]),
Span('kujutab', [{'id': 17, 'form': 'kujutab', 'lemma': 'kujutama', 'upostag': 'VERB', 'xpostag': 'V', 'feats': OrderedDict([('Mood', 'Ind'), ('Number', 'Sing'), ('Person', '3'), ('Tense', 'Pres'), ('VerbForm', 'Fin'), ('Voice', 'Act')]), 'head': 10, 'deprel': 'ccomp', 'deps': '_', 'misc': '_'}]),
Span(',', [{'id': 18, 'form': ',', 'lemma': ',', 'upostag': 'PUNCT', 

---